In [21]:
import datetime as DT
from datetime import date, timedelta
import sys
import pandas as pd
import numpy as np
import json
import pyhive
from pyhive import presto
import inflect
import markdown
from numerize import numerize

In [22]:

# get login credentials from cred.json file
with open('cred.json') as json_file:
    cred = json.load(json_file)
    
username = cred['API_USER']
password = cred['API_PW']

In [23]:

# set presto connection
presto_conn = presto.connect(
            host='presto-gateway.corp.mongodb.com',
            catalog='awsdatacatalog',
            port=443,
            protocol='https',
            username=cred['API_USER'],
            password=cred['API_PW']
        )


In [24]:
# Specify the start and end date for your iteration
start_date = date(2023, 7, 29)
end_date = date.today()  # This gets the current date

# Create an empty list to store the results for each date
results = []

# Loop through dates
current_date = start_date
while current_date <= end_date:
    # Format the current date as a string in 'YYYY-MM-DD' format
    current_date_str = current_date.strftime('%Y-%m-%d')
    
    # Define the SQL query in a structured way using triple quotes
    sql_query = f"""
    with base as (
        SELECT
            org_id,
            properties__context,
            properties__action,
            first_action_ts,
            rank_action
        FROM (
            SELECT
                org_id,
                properties__context,
                properties__action,
                first_action_ts,
                ROW_NUMBER() OVER (PARTITION BY org_id ORDER BY first_action_ts) AS rank_action
            FROM (
                SELECT
                    org_id,
                    properties__context,
                    properties__action,
                    MIN(ts) AS first_action_ts
                FROM (
                    (SELECT DISTINCT
                        integrations__amplitude__groups__organization AS org_id,
                        timestamp AS ts,
                        properties__context AS properties__context,
                        (properties__action) AS properties__action
                    FROM awsdatacatalog.raw_segment.cloud__segment__production_website_full_history fe
                    WHERE
                        fe.integrations__amplitude__groups__organization IS NOT NULL
                        AND timestamp_transformed = DATE'{current_date_str}'
                        AND properties__context='Search Tester - Query Syntax Editor'
                        AND properties__action='Clicked Search')
                    UNION ALL
                    (SELECT DISTINCT
                        integrations__amplitude__groups__organization AS org_id,
                        timestamp AS ts,
                        properties__context AS properties__context,
                        (properties__action) AS properties__action
                    FROM awsdatacatalog.raw_segment.cloud__segment__production_website_full_history fe
                    WHERE
                        fe.integrations__amplitude__groups__organization IS NOT NULL
                        AND timestamp_transformed = DATE'{current_date_str}'
                        AND properties__context='Search Indexes Page - Search Tester'
                        AND properties__action='Clicked Search')
                ) acts
                GROUP BY 1, 2, 3
            ) agg
        ) fn
        WHERE rank_action = 1
    )


    ,deploy as (select
                granularity_id as org_id,
                stage as event,
                cast(ds as timestamp) AS timestamp
            from awsdatacatalog.ns__analytics_postprocessing.analytics__atlas_search_acquisition_funnel_fct
            where stage = 'deployment'
                  and granularity = 'organization')

    ,acquire as (select distinct granularity_id as org_id,
                'acquisition' as event,
                cast(ds as timestamp) as timestamp
            from awsdatacatalog.ns__analytics_postprocessing.analytics__atlas_search_usage_dim
            where granularity = 'organization'
                  and days_since_acquisition=0)

    ,activate as (select distinct granularity_id as org_id,
                'activation' as event,
                min(CAST(ds AS timestamp)) AS timestamp
            from awsdatacatalog.ns__analytics_postprocessing.analytics__atlas_search_usage_dim
            where granularity = 'organization'
                AND lness_30d >= 10
            group by 1,2)

    ,qualify as (SELECT DISTINCT
                X_CLUSTER.org_id,
                'qualification' as event,
                MIN(CAST(ds AS timestamp)) AS timestamp
            FROM awsdatacatalog.ns__analytics_postprocessing.analytics__atlas_search_usage_dim X_SEARCH
            JOIN awsdatacatalog.xform_cloud.cloud_cluster_org_dateversioned_v X_CLUSTER
            ON (
                X_SEARCH.granularity_id = X_CLUSTER.cluster_id
                AND X_SEARCH.ds = X_CLUSTER.partition_date - INTERVAL '1' DAY
            )
            WHERE
                granularity = 'cluster'
                AND lness_30d >= 21
                AND active_7d_usage >= 300
                AND instance_size NOT IN ('SERVERLESS_V2','M0','M2','M5')
            GROUP BY 1,2)
        
    select   fn.date_action
             ,fn.properties__context  
             ,fn.properties__action
             ,fn.deployed
             ,fn.acquired
             ,fn.activated
             ,fn.qualified
             ,count(fn.org_id) as orgs
             ,count(distinct fn.org_id) as orgs_unique
    from       
    (select b.org_id
          ,b.properties__context
          ,b.properties__action
          ,cast(b.first_action_ts as date) as date_action
          ,b.first_action_ts
          ,(de.timestamp) as deployed_ts
          ,case when (de.timestamp) < b.first_action_ts then 1 else 0 end deployed
          ,(a.timestamp) as acquired_ts
          ,case when (a.timestamp) < b.first_action_ts then 1 else 0 end acquired
          ,(act.timestamp) as activated_ts
          ,case when (act.timestamp) < b.first_action_ts then 1 else 0 end activated
          ,(q.timestamp) as qualified_ts
          ,case when (q.timestamp) < b.first_action_ts then 1 else 0 end qualified

              from base b

             left join deploy de
             on b.org_id = de.org_id 

             left join acquire a
             on b.org_id = a.org_id 

             left join activate act
             on b.org_id = act.org_id 

             left join qualify q
             on b.org_id = q.org_id) fn

    group by 1,2,3,4,5,6,7
    order by date_action

    """

    # Execute the SQL query for the current date
    exp_df = pd.read_sql(sql_query, presto_conn)
    
    # Append the results to the list
    results.append(exp_df)

    # Move to the next date
    current_date += timedelta(days=1)


/var/folders/_c/940vv3dx2dd01p8_9hqypt2r0000gn/T/ipykernel_72149/2600501329.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_df = pd.read_sql(sql_query, presto_conn)
/var/folders/_c/940vv3dx2dd01p8_9hqypt2r0000gn/T/ipykernel_72149/2600501329.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_df = pd.read_sql(sql_query, presto_conn)
/var/folders/_c/940vv3dx2dd01p8_9hqypt2r0000gn/T/ipykernel_72149/2600501329.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_df = pd.read_sql(sql_query, presto_conn)
/var

/var/folders/_c/940vv3dx2dd01p8_9hqypt2r0000gn/T/ipykernel_72149/2600501329.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_df = pd.read_sql(sql_query, presto_conn)
/var/folders/_c/940vv3dx2dd01p8_9hqypt2r0000gn/T/ipykernel_72149/2600501329.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_df = pd.read_sql(sql_query, presto_conn)
/var/folders/_c/940vv3dx2dd01p8_9hqypt2r0000gn/T/ipykernel_72149/2600501329.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_df = pd.read_sql(sql_query, presto_conn)
/var

/var/folders/_c/940vv3dx2dd01p8_9hqypt2r0000gn/T/ipykernel_72149/2600501329.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_df = pd.read_sql(sql_query, presto_conn)
/var/folders/_c/940vv3dx2dd01p8_9hqypt2r0000gn/T/ipykernel_72149/2600501329.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_df = pd.read_sql(sql_query, presto_conn)
/var/folders/_c/940vv3dx2dd01p8_9hqypt2r0000gn/T/ipykernel_72149/2600501329.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_df = pd.read_sql(sql_query, presto_conn)
/var

/var/folders/_c/940vv3dx2dd01p8_9hqypt2r0000gn/T/ipykernel_72149/2600501329.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_df = pd.read_sql(sql_query, presto_conn)
/var/folders/_c/940vv3dx2dd01p8_9hqypt2r0000gn/T/ipykernel_72149/2600501329.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_df = pd.read_sql(sql_query, presto_conn)
/var/folders/_c/940vv3dx2dd01p8_9hqypt2r0000gn/T/ipykernel_72149/2600501329.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_df = pd.read_sql(sql_query, presto_conn)
/var

In [25]:
# Concatenate the list of DataFrames into a single DataFrame
combined_df = pd.concat(results)

# Save the combined DataFrame to a CSV file
combined_df.to_csv('results.csv', index=False)  # Change 'results.csv' to your desired file path